regular run:  
`actor.run()` --> done  
regular run with a timer in the iterator:  
`actor.iterator = EndlessViewIterator` --> done

set max tasks  
`actor.run(max_tasks=2)`  --> in branch SPD-410  
see only 2 tasks being taken and run

stop after elapsed time  
`actor.run(stop_function=actor.time_elapsed, elapsed=11)` --> in branch SPD-410 (through run() istead of iterator)  
see that picas doesnt start a new token after 11 seconds of processing

stop when you expect to run out of time  
`actor.run(max_time=1, avg_time_factor=0.9)`  --> in branch SPD-410  
add 3 tokens that sleep for 0.9 seconds and see that picas stops after 1.

not resetting the token when killing picas  
`super(ExampleActor, self).__init__(db, view=view, token_reset_values=None, **viewargs)`  --> in branch SPD-409  
the opposite is the default: killing picas resets the token automatically, also needs to be shown 

# PiCaS examples

## Pushing work to the database
We need to push tokens or tasks to the CouchDB instance, so that PiCaS can fetch the work and execute it one by one.
To accomplish this, we define some functions up next, that will push lines from an input file as commands to tokens. Each line becomes a single token.

In [ ]:
import os
import random
import sys
import time

import couchdb
import picasconfig

In [ ]:
def getNextIndex():
    """Function to set the index sequentially, instead of default random string"""
    db = get_db()
    index = 0
    while db.get(f"token_{index}") is not None:
        index+=1

    return index

def loadTokens(db):
    """Create the tokens from the given file and push to the server"""
    tokens = []
    tokensfile = '/home/lodewijkn/picasclient/examples/quickExample.txt' # put your own path here
    with open(tokensfile) as f:
        input = f.read().splitlines()

    i = getNextIndex()
    for fractal in input:
        token = {
            '_id': 'token_' + str(i),
            'type': 'token',
            'lock': 0,
            'done': 0,
            'hostname': '',
            'scrub_count': 0,
            'input': fractal,
            'exit_code': ''
        }
        tokens.append(token)
        i = i +1
    db.update(tokens)

def get_db():
    """Fetch the server instance"""
    server = couchdb.Server(picasconfig.PICAS_HOST_URL)
    username = picasconfig.PICAS_USERNAME
    pwd = picasconfig.PICAS_PASSWORD
    server.resource.credentials = (username,pwd)
    db = server[picasconfig.PICAS_DATABASE]
    return db

print(f"Pushing tokens to {picasconfig.PICAS_DATABASE} at {picasconfig.PICAS_HOST_URL}")
#Create a connection to the server
db = get_db()
#Load the tokens to the database
loadTokens(db)
print("Tokens have been pushed.")

## Processing tasks stored in the database using PiCaS classes
Next, we define a custom class that is based (inherited) on the RunActor. We need to define the "process_task" method to define how each token is processed. The automation is then taken case of in the base class implementation.  
Of course, you are free to overwrite more parts of the class in case that is needed.  

In [ ]:
from picas.actors import RunActor
from picas.clients import CouchDB
from picas.iterators import TaskViewIterator, EndlessViewIterator
from picas.modifiers import BasicTokenModifier
from picas.executers import execute
from picas.util import Timer

class ExampleActor(RunActor):
    """
    The ExampleActor is the custom implementation of a RunActor that the user needs for the processing.
    Feel free to adjust to whatever you need, a template can be found at: example-template.py
    """
    def __init__(self, db, modifier, view="todo", **viewargs):
        super().__init__(db, view=view, **viewargs)
        self.timer = Timer()
        self.modifier = modifier
        self.client = db

    def process_task(self, token):
        # Print token information
        print("-----------------------")
        print("Working on token: " +token['_id'])
        for key, value in token.doc.items():
            print(key, value)
        print("-----------------------")

        # Start running the main job
        # /usr/bin/time -v ./process_task.sh [input] [tokenid] 2> logs_[token_id].err 1> logs_[token_id].out
        command = "/usr/bin/time -v ./process_task.sh " + "\"" +token['input'] + "\" " + token['_id'] + " 2> logs_" + str(token['_id']) + ".err 1> logs_" + str(token['_id']) + ".out"
        out = execute(command, shell=True)

        ## Get the job exit code in the token
        token['exit_code'] = out[0]
        token = self.modifier.close(token)
        
        # Attach logs in token
        curdate = time.strftime("%d/%m/%Y_%H:%M:%S_")
        try:
            logsout = "logs_" + str(token['_id']) + ".out"
            log_handle = open(logsout, 'rb')
            token.put_attachment(logsout, log_handle.read())

            logserr = "logs_" + str(token['_id']) + ".err"
            log_handle = open(logserr, 'rb')
            token.put_attachment(logserr, log_handle.read())
        except:
            pass


client = CouchDB(url=picasconfig.PICAS_HOST_URL, db=picasconfig.PICAS_DATABASE, username=picasconfig.PICAS_USERNAME, password=picasconfig.PICAS_PASSWORD)
print(f"Connected to the database {picasconfig.PICAS_DATABASE} sucessfully. Now starting work...")
modifier = BasicTokenModifier()

actor = ExampleActor(client, modifier)
actor.run()

Now we want to let the Actor run indefinitely, waiting for work and starting it immediately once its found in the DB.  
Such an Actor should time-out eventually, and for this we define a timer boolean function:

In [ ]:
def time_elapsed(timer, elapsed=30.):
    """
    @param timer: Timer object from the Actor class
    
    @param elapsed: lifetime of the Actor in seconds

    @returns: bool
    """
    return timer.elapsed() > elapsed

This boolean is passed into the EndlessViewIterator's `stop_function`, so the Iterator knows when to stop: when the boolean becomes `True`.

We push some more tokens, and define a new RunActor that will go on indefinitely, except we gave it a `stop_function` to stop after some seconds of waiting.

In [ ]:
print(f"Pushing tokens to {picasconfig.PICAS_DATABASE} at {picasconfig.PICAS_HOST_URL}")
db = get_db()
loadTokens(db)
print("Tokens have been pushed.")

Now we overwrite the Iterator in the Actor, to use an iterator that does not stop, until `stop_function` is called. For this function we use the `time_elapsed` to stop it after 11 seconds, or two scans of the DB for work.

In [ ]:
actor = ExampleActor(client, modifier)
actor.iterator = EndlessViewIterator(actor.iterator, stop_callback=time_elapsed, timer=actor.timer, elapsed=11)
actor.run()